**The Power Method**

Downloading matrix

In [1]:
import requests

matrix_id = "HB/bcsstk16"  # choose one of the IDs above
url = f"https://sparse.tamu.edu/mat/{matrix_id}.mat"
dest_filename = matrix_id.split("/")[-1] + ".mat"

# Download the file using requests
response = requests.get(url, stream=True)
response.raise_for_status()

with open(dest_filename, 'wb') as f:
    for chunk in response.iter_content(chunk_size=8192):
        f.write(chunk)

print(f"Downloaded {dest_filename}")

Downloaded bcsstk16.mat


Loading Matrix

In [2]:
import numpy as np
from scipy.io import loadmat
from scipy.sparse import csr_matrix

# Load the .mat file
data = loadmat("bcsstk16.mat")

# Access the matrix stored in the 'Problem' struct
A = data['Problem'][0, 0]['A']  # This is typically a sparse matrix already

# If A is not in CSR format, convert it
A_sparse = csr_matrix(A)

# Load RHS vector if available
if 'b' in data['Problem'][0, 0].dtype.names:
    b = data['Problem'][0, 0]['b'].flatten()
else:
    b = np.random.rand(A_sparse.shape[0])  # Generate dummy RHS

# Inspect matrix
print("Shape:", A_sparse.shape)
print("Non-zeros:", A_sparse.nnz)

Shape: (4884, 4884)
Non-zeros: 290378


5. The Power Method

    Matrix Requirement: Square

In [3]:
import numpy as np

def power_method(A, max_iter=1000, tol=1e-10):
    # Convert sparse to dense for simplicity
    A = np.array(A.todense(), dtype=float)
    n = A.shape[0]

    # Start with a random vector
    x = np.random.rand(n)
    x = x / np.linalg.norm(x)

    lambda_old = 0.0

    for it in range(max_iter):
        # Multiply by A
        x_new = A @ x

        # Normalize
        x_new_norm = np.linalg.norm(x_new)
        if x_new_norm == 0:
            raise ValueError("Encountered zero vector during iteration.")
        x = x_new / x_new_norm

        # Rayleigh quotient for eigenvalue estimate
        lambda_new = x.T @ (A @ x)

        # Check convergence
        if abs(lambda_new - lambda_old) < tol:
            return lambda_new, x, it+1

        lambda_old = lambda_new

    return lambda_new, x, max_iter

# Example usage
lambda_dom, eigenvector, iterations = power_method(A_sparse)

print("Dominant eigenvalue ≈", lambda_dom)
print("Iterations:", iterations)
print("First 5 entries of eigenvector:", eigenvector[:5])

Dominant eigenvalue ≈ 4943165634.199603
Iterations: 86
First 5 entries of eigenvector: [ 3.10608961e-17 -7.23408212e-18  0.00000000e+00  9.12584390e-17
 -5.20550399e-17]
